# AVANCE 2

## Demostración de Integración del Sistema de Ventas

Este notebook demuestra la integración de los diferentes componentes del sistema:
1. Conexión a la base de datos usando el patrón Singleton
2. Construcción y ejecución de consultas usando el patrón Builder
3. Generación de reportes usando el patrón Factory
4. Ejecución de pruebas unitarias

Primero, importamos las dependencias necesarias:

In [1]:
import pandas as pd
import pytest
from datetime import datetime, timedelta
from IPython.display import display, HTML

# Importar módulos
from src.patterns.database_singleton import DatabaseConnection
from src.patterns.query_builder import SQLQueryBuilder, SalesQueryBuilder
from src.patterns.report_factory import ReportFactory
from src.config import Config

print("Dependencias importadas correctamente.")


Dependencias importadas correctamente.


## 1. Conexión a db (Patrón Singleton)

Demostraremos que siempre obtenemos la misma instancia de conexión y que la conexión es exitosa:

In [2]:
db1 = DatabaseConnection()
db2 = DatabaseConnection()

# Verificar
print("¿Son la misma instancia?:", db1 is db2)

db1.connect()

# Verificar conexión
try:
    connection = db1.get_connection()
    print("\nConexión exitosa a la base de datos")
    print("Parámetros de conexión utilizados:")
    params = Config.DB.get_connection_params()
    # Ocultar la contraseña
    params['password'] = '****'
    for key, value in params.items():
        print(f"  {key}: {value}")
except Exception as e:
    print(f"Error al conectar: {e}")


¿Son la misma instancia?: True

Conexión exitosa a la base de datos
Parámetros de conexión utilizados:
  host: localhost
  user: root
  password: ****
  database: ventas_db
  port: 3306


## 2. Construcción y Ejecución de Consultas (Patrón Builder)

### 2.1 Consulta Básica
Primero, construiremos y ejecutaremos una consulta simple usando el SQLQueryBuilder:


In [ ]:
# Primero veamos si hay datos en la tabla y los rangos de precios
builder = SQLQueryBuilder()
query1, params1 = (
    builder
    .select("COUNT(*) as total_products", "MIN(Price) as min_price", "MAX(Price) as max_price")
    .from_table("products")
    .build()
)

print("1. Verificando datos en la tabla products:")
try:
    df_stats = db1.execute_query(query1, params1)
    print("\nEstadísticas de productos:")
    display(df_stats)
except Exception as e:
    print(f"Error al ejecutar la consulta: {e}")

# Ahora veamos algunos productos con sus precios
builder2 = SQLQueryBuilder()
query2, params2 = (
    builder2
    .select("ProductID", "ProductName", "Price")
    .from_table("products")
    .order_by("Price", "DESC")
    .limit(5)
    .build()
)

print("\n2. Mostrando los 5 productos más caros (sin filtro de precio):")
try:
    df_products = db1.execute_query(query2, params2)
    print("\nResultados:")
    display(df_products)
except Exception as e:
    print(f"Error al ejecutar la consulta: {e}")

# Consulta ajustada con un filtro de precio 
print("\n3. Consulta ajustada con filtro de precio > 90:")
builder3 = SQLQueryBuilder()
query3, params3 = (
    builder3
    .select("ProductID", "ProductName", "Price")
    .from_table("products")
    .where("Price > :min_price", {"min_price": 90})
    .order_by("Price", "DESC")
    .limit(5)
    .build()
)

print("\nConsulta SQL generada:")
print(query3)
print("\nParámetros:")
print(params3)

try:
    df_filtered = db1.execute_query(query3, params3)
    print("\nResultados:")
    display(df_filtered)
except Exception as e:
    print(f"Error al ejecutar la consulta: {e}")


1. Verificando datos en la tabla products:

Estadísticas de productos:


,total_products,min_price,max_price
0,452,0.04,99.88



2. Mostrando los 5 productos más caros (sin filtro de precio):

Resultados:


,ProductID,ProductName,Price
0,98,Shrimp - 31/40,99.88
1,248,Beef - Inside Round,99.32
2,392,Puree - Passion Fruit,98.83
3,345,Bread - Calabrese Baguette,98.60
4,149,Zucchini - Yellow,98.46



3. Consulta ajustada con filtro de precio > 90:

Consulta SQL generada:
SELECT ProductID, ProductName, Price FROM products WHERE Price > %s ORDER BY Price DESC LIMIT 5

Parámetros:
[90]

Resultados:


,ProductID,ProductName,Price
0,98,Shrimp - 31/40,99.88
1,248,Beef - Inside Round,99.32
2,392,Puree - Passion Fruit,98.83
3,345,Bread - Calabrese Baguette,98.60
4,149,Zucchini - Yellow,98.46


### 2.2 Consulta Especializada de Ventas
Ahora usaremos el SalesQueryBuilder para una consulta más específica del dominio:

In [ ]:
# probar una consulta más simple primero
builder = SQLQueryBuilder()
query, params = (
    builder
    .select("COUNT(*) as total_sales")
    .from_table("sales")
    .build()
)

print("Verificando si hay datos en la tabla sales:")
print("\nConsulta SQL:")
print(query)
print("\nParámetros:")
print(params)

try:
    df = db1.execute_query(query, params)
    print("\nResultados:")
    display(df)
except Exception as e:
    print(f"Error al ejecutar la consulta: {e}")

# consultas más complejas
if 'df' in locals() and not df.empty:
    print("\nProcediendo con las consultas detalladas...")
    
    # 1. Consulta agrupada por producto
    sales_builder = SalesQueryBuilder()
    start_date = datetime.now() - timedelta(days=30)
    
    query_products, params_products = (
        sales_builder
        .with_date_range(start_date, datetime.now())
        .group_by_product()
        .build()
    )
    
    print("\n1. Consulta de ventas por producto:")
    print(query_products)
    print("\nParámetros:")
    print(params_products)
    
    try:
        df_products = db1.execute_query(query_products, params_products)
        print("\nResultados de ventas por producto:")
        display(df_products)
    except Exception as e:
        print(f"Error al ejecutar la consulta: {e}")
        df_products = pd.DataFrame()  
    
    # 2. Consulta agrupada por empleado
    sales_builder = SalesQueryBuilder()
    query_employees, params_employees = (
        sales_builder
        .with_date_range(start_date, datetime.now())
        .group_by_employee()
        .build()
    )
    
    print("\n2. Consulta de ventas por empleado:")
    print(query_employees)
    print("\nParámetros:")
    print(params_employees)
    
    try:
        df_employees = db1.execute_query(query_employees, params_employees)
        print("\nResultados de ventas por empleado:")
        display(df_employees)
    except Exception as e:
        print(f"Error al ejecutar la consulta: {e}")
        df_employees = pd.DataFrame()  
    
    # datos para los reportes
    sales_data = {
        'product': df_products.to_dict('records') if not df_products.empty else [],
        'employee': df_employees.to_dict('records') if not df_employees.empty else []
    }


Verificando si hay datos en la tabla sales:

Consulta SQL:
SELECT COUNT(*) as total_sales FROM sales

Parámetros:
[]

Resultados:


,total_sales
0,50000



Procediendo con las consultas detalladas...

1. Consulta de ventas por producto:
SELECT p.ProductID, p.ProductName, COUNT(*) as total_sales, SUM(Quantity) as total_quantity, SUM(TotalPrice) as total_revenue FROM sales INNER JOIN products p ON sales.ProductID = p.ProductID WHERE SalesDate BETWEEN %s AND %s GROUP BY p.ProductID, p.ProductName

Parámetros:
[datetime.datetime(2025, 5, 9, 10, 13, 15, 253080), datetime.datetime(2025, 6, 8, 10, 13, 15, 253080)]

Resultados de ventas por producto:


,ProductID,ProductName,total_sales,total_quantity,total_revenue
0,406,Shrimp - Baby; Warm Water,100,1271,14524.00
1,377,Wine - Red; Cooking,101,1260,15036.00
2,351,Hot Chocolate - Individual,106,1347,16475.00
3,417,Lamb - Pieces; Diced,111,1538,19495.00
4,147,Cheese - Mozzarella,98,1244,13755.00
...,...,...,...,...,...
447,146,Bananas,119,1486,19195.00
448,21,Kiwi,75,960,12499.00
449,281,Bar Mix - Pina Colada; 355 Ml,101,1361,16056.00
450,248,Beef - Inside Round,93,1294,15488.00



2. Consulta de ventas por empleado:
SELECT e.EmployeeID, e.FirstName, e.LastName, COUNT(*) as total_sales, SUM(TotalPrice) as total_revenue FROM sales INNER JOIN employees e ON sales.SalesPersonID = e.EmployeeID WHERE SalesDate BETWEEN %s AND %s GROUP BY e.EmployeeID, e.FirstName, e.LastName

Parámetros:
[datetime.datetime(2025, 5, 9, 10, 13, 15, 253080), datetime.datetime(2025, 6, 8, 10, 13, 15, 704655)]

Resultados de ventas por empleado:


,EmployeeID,FirstName,LastName,total_sales,total_revenue
0,7,Chadwick,Cook,2123,195111.00
1,10,Jean,Vang,2179,284590.00
2,16,Chadwick,Walton,2205,458800.00
3,23,Janet,Flowers,2190,665091.00
4,13,Katina,Marks,2177,368511.00
5,2,Christine,Palmer,2252,58174.00
6,1,Nicole,Fuller,2161,28287.00
7,12,Lindsay,Chen,2168,338664.00
8,8,Julie,Dyer,2273,241680.00
9,5,Desiree,Stuart,2169,140985.00


## 3. Generación de Reportes (Patrón Factory)

Usaremos el ReportFactory para generar diferentes tipos de reportes a partir de los datos obtenidos:

In [5]:
# Verificar
if not sales_data['product'] and not sales_data['employee']:
    print("No hay datos de ventas disponibles para generar reportes.")
else:
    # Generar diferentes tipos de reportes
    report_types = ['sales', 'product', 'employee']
    
    for report_type in report_types:
        print(f"\nGenerando reporte de tipo: {report_type}")
        try:
            data = sales_data.get(report_type, sales_data.get('product', []))
            
            # Crear reporte usando la factory
            report = ReportFactory.create_report(
                report_type, 
                data,
                start_date=start_date,
                end_date=datetime.now()
            )
            
            # Generar el reporte
            result = report.generate()
            
            print("\nResultados del reporte:")
            for key, value in result.items():
                if isinstance(value, (list, dict)):
                    print(f"\n{key}:")
                    if isinstance(value, list) and value:
                        display(pd.DataFrame(value))
                    elif isinstance(value, dict):
                        display(pd.Series(value))
                else:
                    print(f"{key}: {value}")
                    
        except Exception as e:
            print(f"Error al generar el reporte: {e}")
            import traceback
            print("\nDetalles del error:")
            print(traceback.format_exc())



Generando reporte de tipo: sales

Resultados del reporte:
tipo: Reporte de Ventas

periodo:


inicio    2025-05-09 10:13:15
fin       2025-06-08 10:13:23
dtype: object


metricas:


total_ventas               452
ingresos_totales    7863619.00
productos_unicos           452
dtype: object


Generando reporte de tipo: product

Resultados del reporte:
tipo: Reporte de Productos

productos_top:


,producto_id,nombre,cantidad_vendida,ingresos_totales
0,177,Coconut - Shredded; Sweet,1896,23871.00
1,250,Soup - Campbells; Beef Barley,1858,23726.00
2,422,Garlic - Primerba; Paste,1937,23566.00
3,287,Bread - Italian Roll With Herbs,1916,23074.00
4,65,Brandy - Bar,1904,22977.00
5,212,Curry Paste - Madras,1811,22663.00
6,383,Cake - Box Window 10x10x2.5,1882,22202.00
7,156,Sprouts - Alfalfa,1910,22182.00
8,118,Garbag Bags - Black,1641,21849.00
9,424,Vinegar - Tarragon,1830,21740.00


total_productos: 452

Generando reporte de tipo: employee

Resultados del reporte:
tipo: Reporte de Empleados

empleados:


,empleado_id,nombre,ventas_totales,ingresos_generados
0,7,Chadwick Cook,2123,195111.00
1,10,Jean Vang,2179,284590.00
2,16,Chadwick Walton,2205,458800.00
3,23,Janet Flowers,2190,665091.00
4,13,Katina Marks,2177,368511.00
5,2,Christine Palmer,2252,58174.00
6,1,Nicole Fuller,2161,28287.00
7,12,Lindsay Chen,2168,338664.00
8,8,Julie Dyer,2273,241680.00
9,5,Desiree Stuart,2169,140985.00


total_empleados: 23


## 4. Ejecución de unit Test

Ejecutaremos los test directamente en el notebook para verificar que todo funciona correctamente:

In [ ]:
from src.patterns.database_singleton import DatabaseConnection
from src.patterns.query_builder import SQLQueryBuilder, SalesQueryBuilder
from src.patterns.report_factory import ReportFactory
from datetime import datetime, timedelta

def test_singleton_instance():
    """Verifica que se obtiene la misma instancia."""
    db1 = DatabaseConnection()
    db2 = DatabaseConnection()
    assert db1 is db2, "Las instancias deberían ser la misma"
    print("✅ Test singleton_instance: Pasó")

def test_basic_query_builder():
    """Prueba la construcción básica de consultas."""
    builder = SQLQueryBuilder()
    query, params = (
        builder
        .select("ProductID", "ProductName")
        .from_table("products")
        .where("CategoryID = %s", {"category": 1})
        .build()
    )
    
    print("\nConsulta generada:", query)
    print("Parámetros:", params)
    
    assert "SELECT ProductID, ProductName" in query, "La consulta debería incluir las columnas correctas"
    assert "FROM products" in query, "La consulta debería incluir la tabla correcta"
    assert "WHERE CategoryID = %s" in query, "La consulta debería incluir la condición WHERE"
    assert 1 in params, "Los parámetros deberían estar en la lista"
    print("✅ Test basic_query_builder: Pasó")

def test_sales_query_builder():
    """Prueba el builder especializado para consultas de ventas."""
    builder = SalesQueryBuilder()
    start_date = datetime.now() - timedelta(days=30)
    query, params = (
        builder
        .with_date_range(start_date, datetime.now())
        .with_product_category(1)
        .group_by_product()
        .build()
    )
    
    print("\nConsulta de ventas generada:", query)
    print("Parámetros:", params)
    
    assert "FROM sales" in query, "La consulta debería usar la tabla sales"
    assert "JOIN products p" in query, "La consulta debería incluir el JOIN con products"
    assert "SalesDate BETWEEN %s AND %s" in query, "La consulta debería incluir el filtro de fechas"
    assert "CategoryID = %s" in query, "La consulta debería incluir el filtro de categoría"
    assert len(params) == 3, "Deberían haber 3 parámetros (start_date, end_date, category_id)"
    assert 1 in params, "El category_id debería estar en los parámetros"
    print("✅ Test sales_query_builder: Pasó")

print("Ejecutando pruebas unitarias...\n")

# Ejecutar
try:
    test_singleton_instance()
    test_basic_query_builder()
    test_sales_query_builder()
    print("\n✅ Todas las pruebas pasaron exitosamente!")
except AssertionError as e:
    print(f"\n❌ Falló una prueba: {str(e)}")
except Exception as e:
    print(f"\n⚠️ Error al ejecutar las pruebas: {str(e)}")


Ejecutando pruebas unitarias...

✅ Test singleton_instance: Pasó

Consulta generada: SELECT ProductID, ProductName FROM products WHERE CategoryID = %s
Parámetros: [1]
✅ Test basic_query_builder: Pasó

Consulta de ventas generada: SELECT p.ProductID, p.ProductName, COUNT(*) as total_sales, SUM(Quantity) as total_quantity, SUM(TotalPrice) as total_revenue FROM sales INNER JOIN products p ON sales.ProductID = p.ProductID INNER JOIN products p ON sales.ProductID = p.ProductID WHERE SalesDate BETWEEN %s AND %s AND p.CategoryID = %s GROUP BY p.ProductID, p.ProductName
Parámetros: [datetime.datetime(2025, 5, 9, 10, 13, 32, 245206), datetime.datetime(2025, 6, 8, 10, 13, 32, 245206), 1]
✅ Test sales_query_builder: Pasó

✅ Todas las pruebas pasaron exitosamente!


# AVANCE 3: Consultas SQL Avanzadas

En este avance implementaremos consultas SQL avanzadas utilizando:
- Common Table Expressions (CTEs)
- Funciones ventana (ROW_NUMBER, RANK, DENSE_RANK, LAG)
- Análisis temporal y segmentación

Las consultas se ejecutarán usando SQLAlchemy para una mejor integración con pandas.

In [5]:
# Configurar SQLAlchemy
from sqlalchemy import create_engine, text
from src.config import Config

# Obtener parámetros de conexión
params = Config.DB.get_connection_params()

# Crear engine de SQLAlchemy
engine = create_engine(
    f"mysql+mysqlconnector://{params['user']}:{params['password']}@"
    f"{params['host']}:{params['port']}/{params['database']}"
)

print("Conexión SQLAlchemy establecida correctamente.")


Conexión SQLAlchemy establecida correctamente.


## 1. Top Productos por Categoría

Esta consulta utiliza:
- CTE para calcular ventas totales por producto
- ROW_NUMBER() para rankear productos dentro de cada categoría
- Agrupación y ordenamiento para mostrar los top 3 productos de cada categoría

In [6]:
# Consulta 1: Top productos por categoría
query_top_products = """
WITH ProductSales AS (
    -- CTE para calcular ventas totales por producto
    SELECT 
        p.ProductID,
        p.ProductName,
        c.CategoryName,
        SUM(s.Quantity) as TotalQuantity,
        SUM(s.TotalPrice) as TotalRevenue,
        -- Función ventana para rankear productos dentro de cada categoría
        ROW_NUMBER() OVER (
            PARTITION BY c.CategoryID 
            ORDER BY SUM(s.TotalPrice) DESC
        ) as RankInCategory
    FROM products p
    JOIN categories c ON p.CategoryID = c.CategoryID
    JOIN sales s ON p.ProductID = s.ProductID
    GROUP BY p.ProductID, p.ProductName, c.CategoryName, c.CategoryID
)
-- Seleccionar solo los top 3 productos de cada categoría
SELECT 
    CategoryName,
    ProductName,
    TotalQuantity,
    ROUND(TotalRevenue, 2) as TotalRevenue,
    RankInCategory
FROM ProductSales
WHERE RankInCategory <= 3
ORDER BY CategoryName, RankInCategory;
"""

# Ejecutar la consulta
df_top_products = pd.read_sql_query(query_top_products, engine)

print("Top 3 productos más vendidos por categoría:")
display(df_top_products)


Top 3 productos más vendidos por categoría:


,CategoryName,ProductName,TotalQuantity,TotalRevenue,RankInCategory
0,Beverages,Garlic - Primerba; Paste,1937.0,23566.0,1
1,Beverages,Halibut - Steaks,1540.0,20606.0,2
2,Beverages,Soup - Campbells; Cream Of,1605.0,20252.0,3
3,Cereals,Coconut - Shredded; Sweet,1896.0,23871.0,1
4,Cereals,Brandy - Bar,1904.0,22977.0,2
5,Cereals,Vinegar - Tarragon,1830.0,21740.0,3
6,Confections,Soup - Campbells; Beef Barley,1858.0,23726.0,1
7,Confections,Sprouts - Alfalfa,1910.0,22182.0,2
8,Confections,Bread - Raisin Walnut Oval,1641.0,21503.0,3
9,Dairy,Bandage - Flexible Neon,1520.0,20552.0,1


## 2. Análisis de Patrones de Compra de Clientes

Esta consulta utiliza:
- Múltiples CTEs para análisis progresivo
- DENSE_RANK() para rankear clientes por gasto
- NTILE() para segmentación de clientes en percentiles
- Análisis temporal por mes
- Segmentación de clientes basada en percentiles de gasto

In [14]:
# Consulta 2: Análisis de patrones de compra de clientes
query_customer_patterns = """
WITH CustomerMonthlyPurchases AS (
    -- CTE para calcular compras mensuales por cliente
    SELECT 
        c.CustomerID,
        CONCAT(c.FirstName, ' ', c.LastName) as CustomerName,
        DATE_FORMAT(s.SalesDate, '%Y-%m') as PurchaseMonth,
        COUNT(*) as NumberOfPurchases,
        SUM(s.TotalPrice) as MonthlySpending
    FROM customers c
    JOIN sales s ON c.CustomerID = s.CustomerID
    GROUP BY c.CustomerID, CustomerName, PurchaseMonth
),
CustomerStats AS (
    -- CTE para calcular estadísticas por cliente
    SELECT 
        CustomerID,
        CustomerName,
        AVG(MonthlySpending) as AvgMonthlySpending,
        MAX(MonthlySpending) as MaxMonthlySpending,
        -- Calcular el ranking de clientes por gasto promedio
        DENSE_RANK() OVER (
            ORDER BY AVG(MonthlySpending) DESC
        ) as CustomerRank,
        -- Calcular el percentil del cliente
        NTILE(100) OVER (
            ORDER BY AVG(MonthlySpending)
        ) as SpendingPercentile
    FROM CustomerMonthlyPurchases
    GROUP BY CustomerID, CustomerName
)
-- Seleccionar los resultados finales
SELECT 
    CustomerName,
    ROUND(AvgMonthlySpending, 2) as AvgMonthlySpending,
    ROUND(MaxMonthlySpending, 2) as MaxMonthlySpending,
    CustomerRank,
    SpendingPercentile,
    CASE 
        WHEN SpendingPercentile > 90 THEN 'Premium'
        WHEN SpendingPercentile > 70 THEN 'High Value'
        WHEN SpendingPercentile > 40 THEN 'Medium Value'
        ELSE 'Standard'
    END as CustomerSegment
FROM CustomerStats
ORDER BY CustomerRank
LIMIT 10;
"""

# Ejecutar la consulta
df_customer_patterns = pd.read_sql_query(query_customer_patterns, engine)

print("Análisis de patrones de compra de los top 10 clientes:")
display(df_customer_patterns)


Análisis de patrones de compra de los top 10 clientes:


,CustomerName,AvgMonthlySpending,MaxMonthlySpending,CustomerRank,SpendingPercentile,CustomerSegment
0,Darcy Bullock,1872.0,1872.0,1,100,Premium
1,Blake Dalton,1848.0,1848.0,2,100,Premium
2,Forrest Morton,1700.0,1700.0,3,100,Premium
3,Curtis Harmon,1650.0,1650.0,4,100,Premium
4,Allison Davies,1615.0,1615.0,5,100,Premium
5,John Gross,1491.0,1491.0,6,100,Premium
6,Aimee Banks,1475.0,1475.0,7,100,Premium
7,Fred Roberts,1475.0,1475.0,7,100,Premium
8,Spencer Booker,1440.0,1440.0,8,100,Premium
9,Jolene Vincent,1426.0,1426.0,9,100,Premium


## 3. Análisis de Desempeño de Empleados

Esta consulta utiliza:
- CTEs para análisis progresivo de ventas
- RANK() para ranking mensual de empleados
- LAG() para comparar con el mes anterior
- Ventana móvil para promedios de 3 meses
- Cálculos de crecimiento y tendencias

In [17]:
# Consulta 3: Análisis de desempeño de empleados
query_employee_analysis = """
WITH MonthlySales AS (
    -- CTE para ventas mensuales por empleado
    SELECT 
        e.EmployeeID,
        CONCAT(e.FirstName, ' ', e.LastName) as EmployeeName,
        DATE_FORMAT(s.SalesDate, '%Y-%m') as SalesMonth,
        COUNT(*) as NumberOfSales,
        SUM(s.TotalPrice) as TotalRevenue,
        AVG(s.TotalPrice) as AvgSaleValue
    FROM employees e
    JOIN sales s ON e.EmployeeID = s.SalesPersonID
    GROUP BY e.EmployeeID, EmployeeName, SalesMonth
),
EmployeePerformance AS (
    -- CTE para métricas de desempeño
    SELECT 
        EmployeeID,
        EmployeeName,
        SalesMonth,
        NumberOfSales,
        TotalRevenue,
        -- Calcular el ranking mensual por ingresos
        RANK() OVER (
            PARTITION BY SalesMonth 
            ORDER BY TotalRevenue DESC
        ) as MonthlyRank,
        -- Calcular la diferencia con el mes anterior
        LAG(TotalRevenue) OVER (
            PARTITION BY EmployeeID 
            ORDER BY SalesMonth
        ) as PrevMonthRevenue,
        -- Calcular el promedio móvil de 3 meses
        AVG(TotalRevenue) OVER (
            PARTITION BY EmployeeID 
            ORDER BY SalesMonth
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) as Rolling3MonthAvg
    FROM MonthlySales
)
-- Seleccionar los resultados finales
SELECT 
    EmployeeName,
    SalesMonth,
    NumberOfSales,
    ROUND(TotalRevenue, 2) as TotalRevenue,
    MonthlyRank,
    ROUND(
        CASE 
            WHEN PrevMonthRevenue IS NOT NULL AND PrevMonthRevenue > 0
            THEN ((TotalRevenue - PrevMonthRevenue) / PrevMonthRevenue) * 100
            ELSE 0
        END, 
        2
    ) as RevenueGrowth,
    ROUND(Rolling3MonthAvg, 2) as Avg3MonthRevenue
FROM EmployeePerformance
WHERE SalesMonth = (
    SELECT MAX(SalesMonth) FROM EmployeePerformance
)
ORDER BY MonthlyRank;
"""

# Ejecutar la consulta
df_employee_analysis = pd.read_sql_query(query_employee_analysis, engine)

print("Análisis de desempeño de empleados para el último mes:")
display(df_employee_analysis)


Análisis de desempeño de empleados para el último mes:


,EmployeeName,SalesMonth,NumberOfSales,TotalRevenue,MonthlyRank,RevenueGrowth,Avg3MonthRevenue
0,Janet Flowers,2025-06,2190,665091.0,1,0.0,665091.0
1,Tonia Mc Millan,2025-06,2160,631158.0,2,0.0,631158.0
2,Devon Brewer,2025-06,2248,618744.0,3,0.0,618744.0
3,Shelby Riddle,2025-06,2164,569220.0,4,0.0,569220.0
4,Bernard Moody,2025-06,2158,538954.0,5,0.0,538954.0
5,Warren Bartlett,2025-06,2205,512154.0,6,0.0,512154.0
6,Seth Franco,2025-06,2092,463658.0,7,0.0,463658.0
7,Chadwick Walton,2025-06,2205,458800.0,8,0.0,458800.0
8,Kari Finley,2025-06,2112,416940.0,9,0.0,416940.0
9,Wendi Buckley,2025-06,2242,406812.0,10,0.0,406812.0


## Creación y Prueba de Triggers

Vamos a crear un trigger que valide el stock disponible antes de una venta.

In [35]:
# Crear trigger para validar stock
import mysql.connector
from src.config import Config

# Configurar conexión
db = mysql.connector.connect(**Config.DB.get_connection_params())
cursor = db.cursor()

# Crear trigger
print("Creando trigger de validación de stock...")
trigger_validate_stock = """
CREATE TRIGGER before_sale_insert 
BEFORE INSERT ON sales
FOR EACH ROW
BEGIN
    DECLARE available_quantity INT;
    
    -- Obtener cantidad disponible
    SELECT Vitalitydays INTO available_quantity
    FROM products 
    WHERE ProductID = NEW.ProductID;
    
    -- Validar stock
    IF available_quantity < NEW.Quantity THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Error: Stock insuficiente para realizar la venta';
    END IF;
END
"""

try:
    # Eliminar trigger si existe
    cursor.execute("DROP TRIGGER IF EXISTS before_sale_insert")
    db.commit()
    
    # Crear nuevo trigger
    cursor.execute(trigger_validate_stock)
    db.commit()
    print("Trigger creado exitosamente")
    
    # Probar el trigger
    print("\nProbando el trigger:")
    
    # 1. Verificar stock actual del producto 3 (Onions - Cippolini)
    cursor.execute("SELECT ProductID, ProductName, Vitalitydays FROM products WHERE ProductID = 3")
    product = cursor.fetchone()
    print(f"Stock actual del producto {product[1]}: {product[2]} días")
    
    # Obtener el último SalesID
    cursor.execute("SELECT MAX(SalesID) FROM sales")
    last_sale_id = cursor.fetchone()[0]
    next_sale_id = (last_sale_id or 0) + 1
    
    # 2. Intentar venta con cantidad mayor al stock (debe fallar)
    try:
        print("\nIntentando venta con cantidad = 150 (mayor que el stock)...")
        cursor.execute("""
            INSERT INTO sales (SalesID, SalesDate, CustomerID, ProductID, SalesPersonID, Quantity, TotalPrice, Discount, TransactionNumber)
            VALUES (%s, NOW(), 1, 3, 1, 150, 1371.0, 0, 'TEST001')
        """, (next_sale_id,))
        db.commit()
        print("¡Error! La venta no debería haberse permitido")
    except mysql.connector.Error as err:
        print("Error esperado:", err)
    
    # 3. Intentar venta válida con cantidad = 10
    print("\nIntentando venta válida con cantidad = 10...")
    cursor.execute("""
        INSERT INTO sales (SalesID, SalesDate, CustomerID, ProductID, SalesPersonID, Quantity, TotalPrice, Discount, TransactionNumber)
        VALUES (%s, NOW(), 1, 3, 1, 10, 91.40, 0, 'TEST002')
    """, (next_sale_id,))
    db.commit()
    print("Venta válida realizada exitosamente")
    
    # 4. Verificar stock actualizado
    cursor.execute("SELECT ProductID, ProductName, Vitalitydays FROM products WHERE ProductID = 3")
    product_after = cursor.fetchone()
    print(f"\nStock actualizado del producto {product_after[1]}: {product_after[2]} días")
    
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    cursor.close()
    db.close()


Creando trigger de validación de stock...
Trigger creado exitosamente

Probando el trigger:
Stock actual del producto Onions - Cippolini: 111 días

Intentando venta con cantidad = 150 (mayor que el stock)...
Error esperado: 1644 (45000): Error: Stock insuficiente para realizar la venta

Intentando venta válida con cantidad = 10...
Venta válida realizada exitosamente

Stock actualizado del producto Onions - Cippolini: 101 días


In [ ]:
# Crear SP para análisis de ventas
import mysql.connector
from src.config import Config

# Configurar conexión
db = mysql.connector.connect(**Config.DB.get_connection_params())
cursor = db.cursor()

# Crear stored procedure
print("Creando stored procedure para análisis de ventas...")
sp_sales_analysis = """
CREATE PROCEDURE analyze_sales_by_category(
    IN start_date DATE,
    IN end_date DATE
)
BEGIN
    -- Validar fechas
    IF start_date > end_date THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Error: La fecha inicial debe ser menor o igual a la fecha final';
    END IF;
    
    -- Análisis de ventas por categoría
    SELECT 
        c.CategoryID,
        c.CategoryName,
        COUNT(DISTINCT s.ProductID) as UniqueProducts,
        SUM(s.Quantity) as TotalQuantity,
        SUM(s.TotalPrice) as TotalRevenue,
        AVG(s.TotalPrice) as AvgSaleAmount,
        MAX(s.TotalPrice) as MaxSaleAmount
    FROM categories c
    LEFT JOIN products p ON c.CategoryID = p.CategoryID
    LEFT JOIN sales s ON p.ProductID = s.ProductID
    WHERE s.SalesDate BETWEEN start_date AND end_date
    GROUP BY c.CategoryID, c.CategoryName
    ORDER BY TotalRevenue DESC;
END
"""

try:
    # Eliminar SP si existe
    cursor.execute("DROP PROCEDURE IF EXISTS analyze_sales_by_category")
    db.commit()
    
    # Crear nuevo SP
    cursor.execute(sp_sales_analysis)
    db.commit()
    print("Stored procedure creado exitosamente")
    
    # Probar el SP
    print("\nProbando el stored procedure:")
    cursor.execute("CALL analyze_sales_by_category('2025-06-01', '2025-06-10')")
    results = cursor.fetchall()
    
    # Mostrar resultados
    print("\nResultados del análisis de ventas por categoría:")
    for row in results:
        print(f"\nCategoría: {row[1]}")
        print(f"Productos únicos vendidos: {row[2]}")
        print(f"Cantidad total vendida: {row[3]}")
        print(f"Ingresos totales: ${row[4]:.2f}")
        print(f"Venta promedio: ${row[5]:.2f}")
        print(f"Venta máxima: ${row[6]:.2f}")
    
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    cursor.close()
    db.close()


Creando stored procedure para análisis de ventas...
Stored procedure creado exitosamente

Probando el stored procedure:

Resultados del análisis de ventas por categoría:

Categoría: Confections
Productos únicos vendidos: 57
Cantidad total vendida: 82385
Ingresos totales: $998526.00
Venta promedio: $158.52
Venta máxima: $575.00

Categoría: Meat
Productos únicos vendidos: 50
Cantidad total vendida: 71993
Ingresos totales: $862283.00
Venta promedio: $157.18
Venta máxima: $575.00

Categoría: Poultry
Productos únicos vendidos: 47
Cantidad total vendida: 68235
Ingresos totales: $815645.00
Venta promedio: $158.19
Venta máxima: $575.00

Categoría: Cereals
Productos únicos vendidos: 45
Cantidad total vendida: 67344
Ingresos totales: $805381.00
Venta promedio: $157.61
Venta máxima: $575.00

Categoría: Produce
Productos únicos vendidos: 43
Cantidad total vendida: 62290
Ingresos totales: $736564.00
Venta promedio: $154.38
Venta máxima: $575.00

Categoría: Beverages
Productos únicos vendidos: 38
Ca